## Configuration

In [14]:
import os
from dotenv import load_dotenv
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from mcp.types import TextContent

load_dotenv()

class YouTubeAPIError(Exception):
    """Custom exception for YouTube API errors"""
    pass

api_key = os.getenv('YOUTUBE_API_KEY')
if not api_key:
    raise YouTubeAPIError("YouTube API key is not set in environment variables")

def get_youtube_client():
    """Get a singleton YouTube API client instance."""
    api_key = os.getenv('YOUTUBE_API_KEY')
    if not api_key:
        raise YouTubeAPIError("YouTube API key is not set in environment variables")
    
    # Cache the client instance (module-level singleton)
    if not hasattr(get_youtube_client, 'client'):
        get_youtube_client.client = build('youtube', 'v3', developerKey=api_key)
    
    return get_youtube_client.client

## YouTube Video Search

In [16]:
arguments =  {
    "query": "AI",
    "max_results": 10,
    "order": "viewCount",
    "published_after": "2025-01-01T00:00:00Z"
  }

youtube = get_youtube_client()

try:
    query = arguments.get("query", "")
    max_results = min(arguments.get("max_results", 25), 50)  # API limit
    order = arguments.get("order", "relevance")
    duration = arguments.get("duration", "medium")
    upload_date = arguments.get("published_after", None)
        
    search_params = {
        'part': 'snippet',
        'q': query,
        'type': 'video',
        'maxResults': max_results,
        'order': order
    }
    
    if duration != "any":
        search_params['videoDuration'] = duration
    if upload_date:
        search_params['publishedAfter'] = upload_date
    
    response = youtube.search().list(**search_params).execute()
    
    results = []
    for item in response.get('items', []):
        description = item['snippet']['description']
        truncated_desc = description[:200] + ('...' if description else '')
        video_info = {
            'video_id': item['id']['videoId'],
            'title': item['snippet']['title'],
            'description': truncated_desc,
            'channel_title': item['snippet']['channelTitle'],
            'published_at': item['snippet']['publishedAt'],
            'thumbnail_url': item['snippet']['thumbnails'].get('default', {}).get('url', '')
        }
        # Create one TextContent per video
        results.append(TextContent(
            type="text",
            text=(f"**{video_info['title']}**\n"
                    f"Channel: {video_info['channel_title']}\n"
                    f"Video ID: {video_info['video_id']}\n"
                    f"Published: {video_info['published_at']}\n"
                    f"Description: {video_info['description']}")
        ))
    
except HttpError as e:
    raise YouTubeAPIError(f"YouTube API error for query '{query}': {e}")
except Exception as e:
    raise YouTubeAPIError(f"Unexpected error for query '{query}': {e}")

In [22]:
results

[TextContent(type='text', text='**Guess The ITALIAN BRAINROT AI Animals Memes Part 1 | Bombardiro Crocodilo, Tralalero Tralala**\nChannel: Quiz Mirage\nVideo ID: DYYDxLIUASE\nPublished: 2025-04-03T16:26:53Z\nDescription: Guess The ITALIAN BRAINROT AI Animals Memes Part 1 | Bombardiro Crocodilo, Tralalero Tralala, Brr Brr Patapim Did you ......', annotations=None, meta=None),
 TextContent(type='text', text='**Grok AI is Exposing Everyone! | Is it Biased? | Elon Musk | Dhruv Rathee**\nChannel: Dhruv Rathee\nVideo ID: 3cZNbwTXixU\nPublished: 2025-03-20T14:46:02Z\nDescription: Learn 20+ AI Tools and supercharge your productivity. Join my AI Masterclass here: ......', annotations=None, meta=None),
 TextContent(type='text', text='**Drive Thru A.I. Robot Prank**\nChannel: Airrack\nVideo ID: t-oLGxzHyWQ\nPublished: 2025-06-30T17:14:17Z\nDescription: Get an exclusive 15% discount on Saily data plans! Use code AIRRACK at checkout. Download Saily app or go to ......', annotations=None, meta=None)

In [13]:
youtube = get_youtube_client()

arguments = {
    "query": "python tutorial",
    "max_results": 10,
    "duration": "short",
    "published_after": "2025-01-01T00:00:00Z"
  }

query = arguments.get("query", "")
max_results = min(arguments.get("max_results", 25), 50)

response = youtube.search().list(
    part='snippet',
    q=query,
    type='video',
    maxResults=max_results
).execute()

In [10]:
response

{'kind': 'youtube#searchListResponse',
 'etag': '9pJ3jeQidI54dSKO20Kxp0Irjqo',
 'nextPageToken': 'CAoQAA',
 'regionCode': 'IN',
 'pageInfo': {'totalResults': 1000000, 'resultsPerPage': 10},
 'items': [{'kind': 'youtube#searchResult',
   'etag': 'cktaRZ4BcxaKMuI25UaJfU_p9e0',
   'id': {'kind': 'youtube#video', 'videoId': '_uQrJ0TkZlc'},
   'snippet': {'publishedAt': '2019-02-18T15:00:08Z',
    'channelId': 'UCWv7vMbMWH4-V0ZXdmDpPBA',
    'title': 'Python Full Course for Beginners',
    'description': 'Learn Python for AI, machine learning, and web development with this beginner-friendly course! Get 6 months of PyCharm ...',
    'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/_uQrJ0TkZlc/default.jpg',
      'width': 120,
      'height': 90},
     'medium': {'url': 'https://i.ytimg.com/vi/_uQrJ0TkZlc/mqdefault.jpg',
      'width': 320,
      'height': 180},
     'high': {'url': 'https://i.ytimg.com/vi/_uQrJ0TkZlc/hqdefault.jpg',
      'width': 480,
      'height': 360}},
    'ch

## Get Video Metrics

In [19]:
response = youtube.videos().list(
            part='snippet,statistics',
            id="_uQrJ0TkZlc"
        ).execute()

items = response.get('items', [])
item = items[0]
item

{'kind': 'youtube#video',
 'etag': 'mizc1jubY7k0_joo9Tr3rXEbQYs',
 'id': '_uQrJ0TkZlc',
 'snippet': {'publishedAt': '2019-02-18T15:00:08Z',
  'channelId': 'UCWv7vMbMWH4-V0ZXdmDpPBA',
  'title': 'Python Full Course for Beginners',
  'description': 'Learn Python for AI, machine learning, and web development with this beginner-friendly course! 🚀 Get 6 months of PyCharm FREE with the coupon in the description!\n\n❤️ Join this channel to get access to perks:\nhttps://www.youtube.com/channel/UCWv7vMbMWH4-V0ZXdmDpPBA/join\n\n🚀 Want to dive deeper?\n- Check out my Python mastery course: https://mosh.link/python-course\n- Subscribe for more awesome Python content: https://goo.gl/6PYaGF\n\n🔗 Related videos \nhttps://youtu.be/yVl_G-F7m8c?si=aoIiT6CyaRRfBsRM\n\n👉 New version available Watch here: https://youtu.be/kqtD5dpn9C8\n\n📕 Get the FREE goodies:\n- Python cheat sheet: https://mosh.link/python-cheatsheet\n- Supplementary materials (spreadsheet): https://github.com/mosh-hamedani/python-supplem